# RASTER TO BUFFER

In [4]:
from osgeo import gdal
import numpy as np
from scipy.spatial import cKDTree
import rasterio

In [5]:
A_path = r"C:\Users\Runker\Desktop\testaaa\A.tif"
B_path = r"C:\Users\Runker\Desktop\testaaa\B.tif"

In [6]:
# 加载栅格数据
with rasterio.open(A_path) as srcA:
    A = srcA.read(1)  # 读取第一个波段
    affine_a = srcA.transform

with rasterio.open(B_path) as srcB:
    B = srcB.read(1)
    affine_b = srcB.transform

In [7]:

# 选择A和B中非0的像元
non_zero_indices_A = np.nonzero(A)
values_A = A[non_zero_indices_A]
x_coords_A, y_coords_A = rasterio.transform.xy(affine_a, non_zero_indices_A[0], non_zero_indices_A[1], offset='center')
coordinates_A_geo = np.vstack([x_coords_A, y_coords_A]).T

non_zero_indices_B = np.nonzero(B)
values_B = B[non_zero_indices_B]
x_coords_B, y_coords_B = rasterio.transform.xy(affine_b, non_zero_indices_B[0], non_zero_indices_B[1], offset='center')
coordinates_B_geo = np.vstack([x_coords_B, y_coords_B]).T


In [12]:
coordinates_B_geo.shape,coordinates_A_geo.shape

((39233619, 2), (892281, 2))

In [17]:
# 使用cKDTree快速计算距离
tree_A = cKDTree(coordinates_A_geo)
tree_B = cKDTree(coordinates_B_geo)

# 创建一个新数组用于保存输出结果，初始化为0
output_image = np.zeros(B.shape, dtype=np.int32)

# 对于A中的每个像元，找到B中距离不超过300米且值不超过A的像元值+5的所有B像元
for value_a, coord_a in zip(values_A, coordinates_A_geo):
    idx_B = tree_B.query_ball_point(coord_a, 300)  # 300米范围内的B像元索引
    for idx in idx_B:
        if values_B[idx] <= value_a + 5:
            # 将符合条件的B像元标记为9999
            output_image[non_zero_indices_B[0][idx], non_zero_indices_B[1][idx]] = 9999

# 创建新的tif文件保存结果
meta = {
    'driver': 'GTiff',
    'dtype': 'int32',
    'nodata': None,
    'width': B.shape[1],
    'height': B.shape[0],
    'count': 1,
    'crs': srcB.crs,
    'transform': affine_b,  # 使用从文件中提取的 transform
    'compress': 'lzw'
}


with rasterio.open(r"C:\Users\Runker\Desktop\testaaa\result12_5.tif", 'w', **meta) as dst:
    dst.write(output_image, 1)
